In [2]:
!pip install langchain langchain_community
!pip install sentence-transformers
!pip install newsdataapi
%pip install -qU langchain-text-splitters
!pip install pinecone-client
!pip install langchain_pinecone
!pip install schedule

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.8/384.8 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1

In [3]:
from google.colab import userdata

In [4]:
import os
import time
import schedule
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from newsdataapi import NewsDataApiClient
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import PromptTemplate
from langchain.llms import HuggingFaceHub
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser



In [5]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

In [6]:
parser=StrOutputParser()

In [7]:
api = NewsDataApiClient(apikey=userdata.get("NEWS_DATA_API"))
response = api.latest_api(q= "exclusive",language='en')

In [8]:
text=[]

In [9]:
if response["status"]=='success':
  for i in response['results']:
    text.append(i['description'])

In [10]:
len(text)

10

In [11]:
for i in response:
  print(i)
  print("XXXXXXXXXXXXXXXXX\n")

status
XXXXXXXXXXXXXXXXX

totalResults
XXXXXXXXXXXXXXXXX

results
XXXXXXXXXXXXXXXXX

nextPage
XXXXXXXXXXXXXXXXX



In [12]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=5,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents(text)

In [42]:
texts[6]

Document(metadata={'text': '3 Assembly: Assembly of Vehicle 3 remains pending. Company focus has been on Vehicles 1 and 2 throughout the quarter.FAU versus GOM Testing: The partnership with FAU for open water testing allows Nauticus to continue testing Vehicle 2 at a reduced cost compared to GOM open water testing. In addition, FAU provides the team with warehousing and shop facilities to perform minor maintenance as needed. The Atlantic Coast provides multiple deepwater locations close to shore containing bottom'}, page_content='3 Assembly: Assembly of Vehicle 3 remains pending. Company focus has been on Vehicles 1 and 2 throughout the quarter.FAU versus GOM Testing: The partnership with FAU for open water testing allows Nauticus to continue testing Vehicle 2 at a reduced cost compared to GOM open water testing. In addition, FAU provides the team with warehousing and shop facilities to perform minor maintenance as needed. The Atlantic Coast provides multiple deepwater locations close 

In [14]:
pc = Pinecone(api_key=userdata.get('PINECONE_API'))

In [15]:
embed=HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [27]:
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API')

index_name = "nullclass-datastore"

vectorstore_from_docs = PineconeVectorStore.from_documents(
    texts,
    index_name=index_name,
    embedding=embed
)

In [29]:
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm = HuggingFaceHub(
  repo_id=repo_id,
  model_kwargs={"temperature": 0.8, "top_k": 10},
  huggingfacehub_api_token=userdata.get('HUGGING_FACE_API')
)

In [30]:
retriever = vectorstore_from_docs.as_retriever()

In [101]:
prompt =ChatPromptTemplate.from_template(
"""
Based on the {context} provided answer the query i short for the latest news. If the
query related information not present in the provided context then use your own database.

Question:{input}
Answer:''
"""
)

In [102]:
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [103]:
retrieval_chain.invoke({"input":"what is the latest news?"})

{'input': 'what is the latest news?',
 'context': [Document(page_content='Dalian Shipbuilding Industry Company (DSIC) of China Shipbuilding. The researchers are developing autonomous ships and other vessels utilizing cutting-edge technology that may be used to gather data in settings where humans ...Full story available on Benzinga.com'),
  Document(page_content='The Duke and Duchess of Sussex will land in Bogota on Thursday for a quasi-royal visit now expected to include a private meeting with President Gustavo Petro.'),
  Document(page_content='EXCLUSIVE: As Lisa Armstrong enjoys a sun-soaked holiday to Los Angeles, a source tells OK! that the make-up artist is using the trip away to heal and has found a new confidence.'),
  Document(page_content='New York, United States, Aug. 13, 2024 (GLOBE NEWSWIRE) -- As per the Latest Report by Straits Research, The global marine propulsion engine market size is projected to grow from USD 37.96 billion in 2024 to USD 46.72 billion by 2032, growi

In [31]:
template = """
You are a friendly chatbot. You give answer based on your knowledge. The provided context is the
updated information. When user asks the question then first search it in context and if the answer is not
present in context then go for your database but no need to mention it and just give the answer irrespective of
source. For now the context contain upto date information and todays news. The context will be updated with the news data every 7 days.
In Output give only and only the answer section.
Context: {context}
Question: {question}
Answer:
"""
prompt = PromptTemplate(
  template=template,
  input_variables=["context", "question"]
)

In [32]:
rag_chain = (
  {"context": retriever,  "question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)

In [37]:
response=rag_chain.invoke("What are your data sources")

In [38]:
m=f"""
{response.split("Answer:")[-1]}
"""
print(m)


  
I rely on the context provided to me and my own database for answering questions. The context is updated every 7 days with the latest news and information.



In [39]:
def all_input_text_to_db(query):
  print("Information getting updated in db")
  text = query
  vectorstore_from_texts = PineconeVectorStore.from_texts(
      text,
      index_name=index_name,
      embedding=embed
  )
  retriever = vectorstore_from_docs.as_retriever()
  rag_chain = (
  {"context": retriever,  "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
  )
  print("DB is up to date")

In [40]:
def new_news_data_to_db():
  print("Information getting updated with new fetched data in db")
  new_text=[]
  new_api_output = api.latest_api(q= "exclusive",language='en')
  if new_api_output["status"]=='success':
    for i in new_api_output['results']:
      new_text.append(i['description'])
    new_texts = text_splitter.create_documents(new_text)
    vectorstore_from_docs = PineconeVectorStore.from_documents(
        new_texts,
        index_name=index_name,
        embedding=embed
    )
  print("New Data fetched Successfully")

In [41]:
query=input("Whats in your mind.... ")
all_input_text=" "
while query != "exit":
  response=rag_chain.invoke(query)
  answer=response.split("Answer: ")[-1]
  tex=f"""{answer}"""
  print(tex)
  all_input_text=all_input_text+" "+query+" "+answer+ " "
  if len(all_input_text)>2000:
    all_input_text_to_db(all_input_text)
    all_input_text=" "
  schedule.every(1).days.do(new_news_data_to_db)
  query=input("User: ")

Whats in your mind.... Who are you
 
I am a helpful chatbot, here to provide answers to the best of my knowledge.
User: I like cristiano ronaldo
 
That's great! Cristiano Ronaldo is a talented soccer player. Is there anything specific you would like to know about him or his career?
User: give me latest news
 
Nauticus Robotics, Inc. (NASDAQ: KITT), a leading innovator in subsea robotics and software, recently announced its financial results for the quarter ended June 30, 2024.
User: exit
